In [1]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression

2021-12-08 19:09:14.168426: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

columns=['path']
for col in col_list:
    columns.append(str(col)+'-2.0')

In [3]:
input_dim = [182, 218, 182]
num_output = len(columns)-1
batch_size=4

In [4]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid')[columns]
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='standard')

scaler_instance = train_gen.get_scaler_instance()

In [5]:
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid')[columns]
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance)

In [6]:
model = SFCN(
    input_dim=[182, 218, 182, 1], 
    output_dim=num_output,
    conv_num_filters=[32, 64, 128, 256, 256, 64], 
    conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
    conv_strides=[1, 1, 1, 1, 1, 1],
    conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
    maxpool_size=[2, 2, 2, 2, 2],
    batch_norm=True,
    dropout=False,
    softmax=False,
    gpu_num=8,
    name='sfcn'
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2021-12-08 19:09:22.427808: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-08 19:09:22.429991: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-08 19:09:22.559280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-08 19:09:22.564932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-08 19:09:22.567964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:07:0

In [7]:
model.compile(learning_rate=3e-4)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [8]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40)

2021-12-08 19:09:33.009912: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_2380"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  va

Epoch 1/40
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


2021-12-08 19:09:57.129277: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-08 19:09:57.210234: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [ ]:
# train_df.iloc[9]['path']

In [ ]:
# train_df['path'].iloc[0]

In [7]:
#model.load_weights('weights/checkpoint_SFCN')
#data_size=200
#X = np.random.normal(loc=5, scale=5, size=(data_size, *(input_dim), 1))
#y = np.random.normal(loc=1, scale=1, size=(data_size, num_output))
#model.train(X, y, batch_size, 300)

In [ ]:
#x_test = np.random.normal(loc=8, scale=3, size=(1, 160, 192, 160, 1))

In [ ]:
#model.predict(x_test)

In [ ]:
#y_test = np.mean(x_test, axis=(1,2,3,4))
#print(y_test)